Workflow:
1. Read all S&P500 tickers from Wikipedia page
2. Store them in a list
3. Download each stock form YF
4. Save all data in a pd dataframe

In [1]:
# import bs4 as bs # beautifulsoup4 to pull data out of HTML files
import requests # Requests to grab source code from Wiki page
import yfinance as yf
import datetime
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [2]:
resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies') # Wiki link with all the tickers 
soup = BeautifulSoup(resp.text, 'lxml') # Reads all text in the link as an html file
table = soup.find('table', {'class': 'wikitable sortable'}) # filter the html file by the table. In wikipedia, all table contents are under the class 'wikitable sortable'.

In [3]:
tickers = []

for row in table.findAll('tr')[1:]: # 'tr' stands for table row. After the header row, grab the ticker and append it to the empty list.
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)
    
tickers = [s.replace('\n', '') for s in tickers]

In [57]:
# Now we import data of the tickers using YF
start = datetime.datetime(2007 ,1 ,1)
end = datetime.datetime(2022, 12, 31)
data = yf.download(tickers, start = start, end = end)

data.index = pd.to_datetime(data.index)
data.index = data.index.strftime('%Y-%m-%d')

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BRK.B: No timezone found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


tickersThe dataframe, 'data' has multi index - two rows for columns. We transform it to make it look better

In [66]:
df = data.stack().reset_index().rename(index = str,  # .stack() moves the 2nd row of the columns to the rows,
                                       columns = {
                                           "level_1":"Symbol"
                                       }).sort_values(['Symbol', 'Date']) #  .reset_index() collapses the two rows of columns into a single one
df.set_index('Date', inplace = True)

In [32]:
import os
cwd = os.getcwd()
cwd

'/Users/skim/Desktop/Code/Quant/StatArb_AvellanedaLee'

In [143]:
# df_dividends.to_csv(r"/Users/skim/Desktop/Code/Quant/data/snp500_dividends.csv")
# df.to_csv(r"/Users/skim/Desktop/Code/Quant/data/snp500.csv")
df_stocks = pd.read_csv(r"/Users/skim/Desktop/Code/Quant/data/snp500.csv")

In [ ]:
ticker_names = np.unique(df_stocks['Symbol'])
df_dividends = pd.DataFrame(0, columns = ticker_names, index = data.index)

for i in ticker_names[0:5]:
    temp_ticker = yf.Ticker(str(i))
    temp_div_data = yf.Ticker(str(i)).history(start = start, end = end)['Dividends']
    temp_div_data.index = pd.to_datetime(temp_div_data.index)
    temp_div_data.index = temp_div_data.index.strftime('%Y-%m-%d')
    df_dividends[i] = temp_div_data

In [91]:
df_stocks

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,2007-01-03,A,22.247175,24.535049,25.379112,24.356222,25.028612,3599291.0
1,2007-01-04,A,22.318521,24.613733,24.749641,23.934193,24.535049,2899033.0
2,2007-01-05,A,22.110973,24.384834,24.606581,24.320457,24.535049,3741887.0
3,2007-01-08,A,22.033134,24.299000,24.377682,24.091558,24.306152,2176966.0
4,2007-01-09,A,22.059086,24.327612,24.549356,24.055794,24.377682,1937908.0
...,...,...,...,...,...,...,...,...
1893708,2022-12-23,ZTS,145.759995,145.759995,145.889999,143.539993,144.509995,1017900.0
1893709,2022-12-27,ZTS,145.300003,145.300003,146.149994,143.570007,145.910004,957900.0
1893710,2022-12-28,ZTS,143.830002,143.830002,146.639999,143.770004,145.179993,1443900.0
1893711,2022-12-29,ZTS,148.149994,148.149994,148.509995,145.139999,145.199997,1298900.0


Now onto treasury yields

In [97]:
from fredapi import Fred

fred = Fred(api_key='5b03dd2d224bde28384e8754080d5bf5')

In [108]:
# get data for 8 different tenors of US treasury
startDate = '2007-01-01'
endDate = '2022-12-31'
df = []
ids = ['DGS{}'.format(i) for i in ['3MO', 1,2,5,7,10,20,30]]
for s in ids:
    df.append(fred.get_series(s, observation_start=startDate, observation_end=endDate)/100)
  
df = pd.concat(df,axis=1)
df.columns = ids
df = df.dropna()
df.index.name = 'Date'

In [118]:
# df.to_csv(r"/Users/skim/Desktop/Code/Quant/data/treasuries.csv")
df_yields = pd.read_csv(r"/Users/skim/Desktop/Code/Quant/data/treasuries.csv")

In [ ]:
df_yields

,Date,DGS3MO,DGS1,DGS2,DGS5,DGS7,DGS10,DGS20,DGS30
0,2007-01-02,0.0507,0.0500,0.0480,0.0468,0.0468,0.0468,0.0487,0.0479
1,2007-01-03,0.0505,0.0498,0.0476,0.0466,0.0466,0.0467,0.0485,0.0477
2,2007-01-04,0.0504,0.0495,0.0471,0.0461,0.0461,0.0462,0.0481,0.0472
3,2007-01-05,0.0505,0.0498,0.0476,0.0465,0.0465,0.0465,0.0484,0.0474
4,2007-01-08,0.0508,0.0501,0.0478,0.0466,0.0466,0.0466,0.0484,0.0474
...,...,...,...,...,...,...,...,...,...
3999,2022-12-23,0.0434,0.0466,0.0431,0.0386,0.0383,0.0375,0.0399,0.0382
4000,2022-12-27,0.0446,0.0475,0.0432,0.0394,0.0393,0.0384,0.0410,0.0393
4001,2022-12-28,0.0446,0.0471,0.0431,0.0397,0.0397,0.0388,0.0413,0.0398
4002,2022-12-29,0.0445,0.0471,0.0434,0.0394,0.0391,0.0383,0.0409,0.0392
